In [7]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh


# right eyes indices
RIGHT_EYE=[33, 160, 158, 133, 154, 144] 

# Left eyes indices 
LEFT_EYE =[362, 385, 387, 263, 373, 380] 

def eye_aspect_ratio(image, landmarks, right_indices, left_indices):
    rh_right = landmarks[right_indices[0]][:2]
    rh_left = landmarks[right_indices[3]][:2]
    rv_top1 = landmarks[right_indices[1]][:2]
    rv_top2 = landmarks[right_indices[2]][:2]
    rv_bottom1 = landmarks[right_indices[5]][:2]
    rv_bottom2 = landmarks[right_indices[4]][:2]
    
    lh_right = landmarks[left_indices[0]][:2]
    lh_left = landmarks[left_indices[3]][:2]
    lv_top1 = landmarks[left_indices[1]][:2]
    lv_top2 = landmarks[left_indices[2]][:2]
    lv_bottom1 = landmarks[left_indices[5]][:2]
    lv_bottom2 = landmarks[left_indices[4]][:2]
    
    rhDistance = abs(rh_right - rh_left)
    rvDistance1 = abs(rv_top1 - rv_bottom1)
    rvDistance2 = abs(rv_top2 - rv_bottom2)
    rvDistance = abs(rvDistance1 + rvDistance2)
    
    lvDistance1 = abs(lv_top1 - lv_bottom1)
    lvDistance2 = abs(lv_top2 - lv_bottom2)
    lhDistance = abs(lh_right - lh_left)
    lvDistance = abs(lvDistance1 + lvDistance2)

    reRatio = rvDistance/(2*rhDistance)
    leRatio = lvDistance/(2*lhDistance)
    Ratio = reRatio + leRatio

    ear = Ratio / 2
    
    return ear

drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
        while cap.isOpened():
            success, image = cap.read()

            if not success:
                print("웹캠 인식 불가")
                continue

            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # image로부터 랜드마크 반환
            results = face_mesh.process(image)
            
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # 반환된 랜드마크를 image에 덧씌움
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    mypose = face_landmarks
                    tmpList = []
                    for idx,lm in enumerate(mypose.landmark):
                        point=np.array([lm.x, lm.y, lm.z])
                        tmpList.append(point)
                    #print(tmpList)
            
                    ear = eye_aspect_ratio(image, tmpList, RIGHT_EYE, LEFT_EYE)
                    print(ear)

            # 얼굴 전체 그물
            mp_drawing.draw_landmarks(
                image=image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_tesselation_style())
                    
            # 얼굴 윤곽선 (눈전체/입/눈썹 포함)
            mp_drawing.draw_landmarks(
                image=image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_CONTOURS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_contours_style())
                    
            # 눈/입/눈썹 
            mp_drawing.draw_landmarks(
                image=image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_IRISES,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_iris_connections_style())
                    
            # selfie화면이기 때문에 좌우반전하여 출력
            cv2.imshow('MediaPipe Face Mesh', cv2.flip(image, 1))
            
            if cv2.waitKey(5) & 0xFF == ord('q'):
                break
cap.release()
cv2.destroyAllWindows()

[0.10262145 8.5116966 ]
[0.09115663 6.80445719]
[0.09711724 5.96119017]
[ 0.11367315 10.81332672]
[0.11119977 8.33834805]
[1.06823687e-01 1.96882809e+02]
[ 0.08392186 27.6822193 ]
[ 0.08091214 17.27546206]
[ 0.08073801 11.06622668]
[ 0.07604918 11.62712896]
[0.07215857 7.34425   ]
[0.07506945 7.69214323]
[ 0.09329129 12.18904053]
[ 0.09737201 65.42136892]
[9.44285550e-02 1.87016153e+02]
[ 0.09853188 16.78687681]
[9.61063723e-02 1.06427757e+02]
[ 0.09230191 72.3514101 ]
[ 0.0916637  13.52856993]
[ 0.09575643 11.04716312]
[0.09434719 8.9039105 ]
[0.09910111 7.07272065]
[ 0.0995297  28.69435068]
[ 0.09688048 10.93248205]
[ 0.09417981 10.21193422]
[0.09279604 9.84771814]
[ 0.09741206 14.84991919]
[ 0.09188638 13.10307693]
[ 0.09220931 12.04684707]
[0.09788586 8.9738721 ]
[0.09704954 6.10004645]
[ 0.09780242 11.73670285]
[ 0.09556223 11.97433878]
[ 0.09353704 11.01684949]
[0.09511022 8.37400586]
[0.09795252 7.92451954]
[ 0.09623289 24.6362365 ]
[0.09344623 9.50075667]
[ 0.0935472  47.562839